# Basic Prompt Structures Tutorial

## Overview

This tutorial explores two essential prompt structures used in AI interactions:
1. Single-turn prompts
2. Multi-turn prompts (conversations)

We'll utilize Gemini Pro 1.5 via OpenRouter and LangChain to illustrate these concepts.

## Motivation

Grasping various prompt structures is essential for effective AI communication. Single-turn prompts excel in quick, direct queries, while multi-turn prompts facilitate more nuanced, context-rich exchanges. Proficiency in these structures enhances the versatility and efficacy of AI applications across diverse scenarios.

## Key Components

1. **Single-turn Prompts**: One-time interactions with the AI model.
2. **Multi-turn Prompts**: Sequential exchanges that preserve context.
3. **Prompt Templates**: Standardized structures for consistent AI querying.
4. **Conversation Chains**: Techniques for maintaining context across multiple interactions.

## Setup

First, let's import the necessary libraries and set up our environment.

In [1]:
from os import getenv
from typing import List

from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI

load_dotenv()


# Initialize the language model
llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    model_name="google/gemini-pro-1.5",
)

## 1. Single-turn Prompts

Single-turn prompts are one-shot interactions with the language model. They consist of a single input (prompt) and generate a single output (response).

In [2]:
single_turn_prompt = "What are the three primary colors?"
print(llm.invoke(single_turn_prompt).content)

The three primary colors are **red**, **yellow**, and **blue**.  These are called primary because they cannot be created by mixing other colors together, but they can be mixed to create a wide range of other colors.



Now, let's use a PromptTemplate to create a more structured single-turn prompt:

In [3]:
structured_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Provide a brief explanation of {topic} and list its three main components.",
)

chain = structured_prompt | llm
print(chain.invoke({"topic": "color theory"}).content)

Color theory is a set of guiding principles that explain how colors mix, match, and create different visual effects. It explores how humans perceive color and the visual effects of how colors are combined, matched, or contrasted.  It's used in art, design, marketing, and other fields to evoke specific moods, create harmony, or make things stand out.

The three main components of color theory are:

1. **The Color Wheel:** A visual representation of colors arranged according to their relationships.  Different color wheels can be used (e.g., RYB or RGB), but they all illustrate basic relationships like complementary, analogous, and triadic colors.
2. **Color Harmony:**  This refers to aesthetically pleasing color combinations.  It's based on the relationships of colors on the color wheel and aims to create a sense of balance and visual appeal.
3. **The Color Context:** This emphasizes how colors appear relative to other colors surrounding them.  The same color can look dramatically differ

You can learn more about LangChain Expression Language from [LCEL](https://python.langchain.com/docs/concepts/lcel/)

## 2. Multi-turn Prompts (Conversations)

Multi-turn prompts involve a series of interactions with the language model, allowing for more complex and context-aware conversations.

In [4]:
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []


# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}


def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | llm

conversation = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
)

print(
    conversation.invoke(
        input={
            "question": "Hi, I'm learning about space. Can you tell me about planets?"
        },
        config={"configurable": {"session_id": "foo"}},
    ).content
)
print(
    conversation.invoke(
        input={"question": "What's the largest planet in our solar system?"},
        config={"configurable": {"session_id": "foo"}},
    ).content
)
print(
    conversation.invoke(
        input={"question": "How does its size compare to Earth?"},
        config={"configurable": {"session_id": "foo"}},
    ).content
)

Let's talk about planets!  Here's a breakdown covering the basics:

**What is a planet?**

According to the International Astronomical Union (IAU), a planet is a celestial body that:

1. **Orbits a star (in our case, the Sun).**
2. **Is massive enough for its own gravity to make it round.**
3. **Has cleared its orbital neighborhood.**  This means it's gravitationally dominant enough to have either absorbed or flung away other objects in its orbital path.

**Types of Planets in Our Solar System:**

Our solar system has eight planets, which are divided into two main categories:

* **Inner, Rocky Planets (Terrestrial Planets):** These are smaller, denser planets made mostly of rock and metal. They are closer to the Sun and have few or no moons.
    * **Mercury:** The smallest planet, closest to the Sun, and very hot.
    * **Venus:** Similar in size to Earth but incredibly hot due to a thick, toxic atmosphere.
    * **Earth:** Our home, the only known planet with life, and has one moon.
 

Let's compare how single-turn and multi-turn prompts handle a series of related questions:

In [6]:
# Single-turn prompts
prompts = [
    "What is the capital of France?",
    "What is its population?",
    "What is the city's most famous landmark?",
]

print("Single-turn responses:")
for prompt in prompts:
    print(f"Q: {prompt}")
    print(f"A: {llm.invoke(prompt).content}\n")

# Multi-turn prompts
print("Multi-turn responses:")
for prompt in prompts:
    print(f"Q: {prompt}")
    response = conversation.invoke(
        input={"question": prompt}, config={"configurable": {"session_id": "bar"}}
    ).content
    print(f"A: {response}\n")

Single-turn responses:
Q: What is the capital of France?
A: Paris


Q: What is its population?
A: Please specify what "it" refers to. I need more context to answer your question.  For example, are you asking about a country, city, state, or something else entirely?


Q: What is the city's most famous landmark?
A: Please tell me which city you're asking about! I need to know the city to tell you its most famous landmark.


Multi-turn responses:
Q: What is the capital of France?
A: Paris


Q: What is its population?
A: The population of Paris is a bit tricky because it depends on what area you're talking about.  The city of Paris proper (intra-muros) has a population of around **2.1 million** people.  However, the Île-de-France region, which includes Paris and its surrounding suburbs (often referred to as the Paris Region or Greater Paris), has a population of over **12 million** people.


Q: What is the city's most famous landmark?
A: The Eiffel Tower is arguably the most famous landmar